In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
filename = '../../savior_bags/13_pics/40.jpg'

In [ ]:
im1 = cv2.imread(filename)

In [ ]:
print ('Array type = %s  shape = %s' % (im1.dtype, im1.shape))

In [ ]:
shape = im1.shape
height, width, depth = shape
print('height: %s  width: %s' % (height, width))

In [ ]:
plt.imshow(im1)
plt.show()

In [ ]:
# copy image
im2 = im1.copy()
# paint square
S = 50
u, v = 100, 500
im2[u:(u+S), v:(v+S), :] = 255

In [ ]:
plt.imshow(im2)
plt.show()

In [ ]:
B = im1[:,:,0]
G = im1[:,:,1]
R = im1[:,:,2] 

In [ ]:
im1_rgb = im1[:,:,::-1]
#or using OpenCV conventions:
# im1_rgb = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
# plt.imshow(im1_rgb); plt.show()

In [ ]:
plt.imshow(im1_rgb); plt.show()


In [ ]:
#CROPPING THE PICTURE

In [ ]:
im1_cropped = im1[130:,:,::-1]
plt.imshow(im1_cropped); plt.show()

In [ ]:
yellow = np.uint8([[[255,255,0 ]]])
hsv_green = cv2.cvtColor(yellow,cv2.COLOR_BGR2HSV)
print hsv_green

In [ ]:
# Convert BGR to HSV
hsv = cv2.cvtColor(im1, cv2.COLOR_BGR2HSV)
plt.imshow(hsv); plt.show()

# define range of blue color in HSV
lower_blue = np.array([0,100,100])
upper_blue = np.array([90,255,255])

# Threshold the HSV image to get only blue colors
mask = cv2.inRange(hsv, lower_blue, upper_blue)
plt.imshow(mask); plt.show()

# Bitwise-AND mask and original image
res = cv2.bitwise_and(im1,im1, mask= mask)

plt.imshow(res); plt.show()
#cv2.imshow('im1',im1)
#cv2.imshow('mask',mask)
#cv2.imshow('res',res)
#k = cv2.waitKey(5) 
    

In [ ]:
bilateral_filtered_image = cv2.bilateralFilter(res, 5, 175, 175)
edge_detected_image = cv2.Canny(bilateral_filtered_image, 75, 200)
plt.imshow(edge_detected_image); plt.show()
cv2.imshow('Edge', edge_detected_image)
cv2.waitKey(0)
#BESSERE VISUALISIERUNG!!! -> UMRANDUNG INS ORIGINALBILD!!!

In [ ]:
bilateral_filtered_image = cv2.bilateralFilter(im1_cropped, 5, 175, 175)
edge_detected_image = cv2.Canny(bilateral_filtered_image, 75, 200)
plt.imshow(edge_detected_image); plt.show()
cv2.imshow('Edge', edge_detected_image)
cv2.waitKey(0)

In [ ]:
_, contours, hierarchy = cv2.findContours(edge_detected_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

contour_list = []
for contour in contours:
    approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
    area = cv2.contourArea(contour)
    if ((len(approx) > 8) & (len(approx) < 23) & (area > 30) ):
        contour_list.append(contour)

cv2.drawContours(im1_cropped, contour_list,  -1, (255,0,0), 2)
cv2.imshow('Objects Detected',im1_cropped)
cv2.waitKey(0)

Where is the transparency?

In [ ]:
im2_bgra = cv2.imread(filename, cv2.IMREAD_UNCHANGED)

In [ ]:
print im2_bgra.shape

In [ ]:
im2_rgba =  cv2.cvtColor(im2_bgra, cv2.COLOR_BGRA2RGBA)

In [ ]:
plt.imshow(im2_rgba); plt.show()

In [ ]:
print im2_rgba.shape

In [ ]:
mask = im2_rgba[:,:,3]

In [ ]:
plt.imshow(mask, cmap='gray'); plt.show()

In [ ]:
plt.imshow(mask[500:700, 500:700], cmap='gray'); plt.show()

In [ ]:
alpha = mask / 255.0
beta = 1.0 - alpha

In [ ]:
background_bgr = cv2.imread('data/bg.jpg')
background_rgb = cv2.cvtColor(background_bgr, cv2.COLOR_BGR2RGB)
H, W, _ = im2_rgba.shape
bg = cv2.resize(background_rgb, (W, H))

plt.imshow(bg); plt.show()

In [ ]:
funky_rgb = np.zeros(dtype='uint8', shape=(height, width, 3))

In [ ]:
for i in [0, 1, 2]:
    funky_rgb[:, :, i] =  im2_rgba[:,:,i] * alpha + bg[:,:,i] * (1-alpha)

In [ ]:
plt.imshow(funky_rgb); plt.show()

In [ ]:
conv = cv2.cvtColor(funky_rgb, cv2.COLOR_RGB2BGR)
cv2.imwrite( 'data/funky_rgb.png', conv)